In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.model_selection import cross_val_score, train_test_split

In [3]:
train_file = "/kaggle/input/digit-recognizer/train.csv"
data = pd.read_csv(train_file)
data.shape

(42000, 785)

In [4]:
X = data.drop(['label'], axis=1).values
y = data['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train = tf.reshape(X_train, [33600,28,28,1])
X_test = tf.reshape(X_test, [8400,28,28,1])
y_train = tf.reshape(y_train, [33600,1])
y_test = tf.reshape(y_test, [8400,1])

Input_shape=(28,28,1) 
print(X_train.shape, y_train.shape)

(33600, 28, 28, 1) (33600, 1)


2022-09-30 19:41:59.866995: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
print(X_train.shape)
model = Sequential((
Conv2D(filters=32,kernel_size=(4,4), strides=1,padding='same',activation='relu',input_shape =(28,28,1)),
MaxPooling2D((2,2)),
Conv2D(filters=64,kernel_size=(4,4), strides=1,padding='same',activation='relu'),
MaxPooling2D((2,2)),
Conv2D(filters=128,kernel_size=(4,4), strides=1,padding='valid',activation='relu'),
MaxPooling2D((2,2)),
Flatten(),
Dense(units=50, activation='relu'),
Dense(units=10, activation='softmax')
))

model.compile(loss=SparseCategoricalCrossentropy(), optimizer='Adam')

(33600, 28, 28, 1)


In [6]:
model.fit(X_train, y_train, epochs=5)

2022-09-30 19:42:00.545919: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
1050/1050 [==============================] - 31s 28ms/step - loss: 0.6114
Epoch 2/5
1050/1050 [==============================] - 30s 28ms/step - loss: 0.1067
Epoch 3/5
1050/1050 [==============================] - 30s 28ms/step - loss: 0.0658
Epoch 4/5
1050/1050 [==============================] - 30s 28ms/step - loss: 0.0452
Epoch 5/5
1050/1050 [==============================] - 30s 28ms/step - loss: 0.0415


In [7]:
model.evaluate(X_train, y_train)

1050/1050 [==============================] - 10s 9ms/step - loss: 0.0237


0.023725520819425583

In [8]:
from sklearn import metrics
train_predictions = pd.DataFrame(model.predict(X_train))
train_predictions = train_predictions.idxmax(axis=1)
metrics.accuracy_score(y_train, train_predictions) 


0.9929464285714286

In [9]:
test_predictions = pd.DataFrame(model.predict(X_test))
test_predictions = test_predictions.idxmax(axis=1)
metrics.accuracy_score(y_test, test_predictions) 


0.9842857142857143

In [10]:
test_file = "/kaggle/input/digit-recognizer/test.csv"
test = pd.read_csv(test_file)
imageid = test.index.values + 1
test = tf.reshape(test, [28000,28,28,1])

In [11]:
predictions = model.predict(test)
predictions = pd.DataFrame(predictions)
predictions = predictions.idxmax(axis=1)

In [12]:
output = pd.DataFrame({'ImageId': imageid, 'Label': predictions},)
output.to_csv('submission.csv', index=False)